# Mestre do Mario Kart
Você e seus amigos estão disputando um campeonato de Mario Kart chamado o “Mestre do Mario Kart”. Neste campeonato,
um jogador começa como “mestre do Mario Kart” e os outros como desafiantes, que enfrentam o mestre a cada rodada. 

- Em
uma rodada, um desafiante joga contra o então “mestre do Mario Kart” e precisa vencer 2 partidas seguidas para se tornar o
novo “mestre do Mario Kart”; 
- caso não consiga, ele retorna ao final da fila de desafiantes. 
- Se o mestre é derrotado, o
desafiante se torna o novo “mestre do Mario Kart” e o perdedor entra no final da fila de desafiantes. 
- Um competidor é
declarado vencedor do campeonato se ele, enquanto “mestre do Mario Kart”, venceu todos os outros competidores
consecutivamente.

## Nível de habilidade

Cada competidor possui um nível de habilidade, que é um número inteiro de 1 a 99. 

Suponha que, em uma partida disputada
entre competidores de níveis $x$ e $y$, a probabilidade de vitória do competidor de nível $x$ é dada por $x/(x+y)$ e a do competidor de
nível $y$ é $y/(x+y)$.

## Solicitação

- Para uma lista de campeonatos possíveis, disponibilizada no arquivo campeonatos.txt, enviado junto ao enunciado do
desafio, pedimos que você nos envie um arquivo no formato .txt com a **probabilidade de que o primeiro “mestre do
Mario Kart” seja o vencedor para cada um dos campeonatos, com precisão de pelo menos 6 casas decimais**, em que
cada linha do arquivo contém a probabilidade do campeonato correspondente. 

- Caso você não consiga determinar a
probabilidade de algum campeonato, pedimos que preencha a linha do campeonato com 0, de forma a facilitar o processo de
correção.

In [1]:
using LinearAlgebra

x = 1
y = 2

pAB = (y / (x + y))^2;
pBA = (x / (x + y))^2;
pAα = 1 - pAB;
pBβ = 1 - pBA;

#M = [(1 - pAB) pAB; pBA (1 - pBA)]
Q = [0 pAB
     pBA 0]
R = [pAα 0
     0 pBβ]
# M = [0 pAB pAα 0
#      pBA 0 0 pBβ
#      0 0 1 0
#      0 0 0 1]

2×2 Matrix{Float64}:
 0.555556  0.0
 0.0       0.888889

In [2]:
N = inv(I - Q)

2×2 Matrix{Float64}:
 1.05195   0.467532
 0.116883  1.05195

In [3]:
N*R

2×2 Matrix{Float64}:
 0.584416   0.415584
 0.0649351  0.935065

In [4]:

# Probability that A(ability x) loses against B(ability y)
# therefore, there will be a probability p^2 
# that B will become the new Mario Kart Master
prob_lose(x, y) = y // (x+y)

prob_lose (generic function with 1 method)

In [5]:
using Combinatorics

struct MarkovState{T<:Integer}
    # Stores the player sequence
    # in which the first element is the Mario Kart Master,
    # the second element is the challenger(first one in the queue)
    # and so on...
    player_sequence::Vector{T}
    # The respective abilities of each player
    ability_sequence::Vector{T}
    # The current amount of times the Mario Kart Master has won
    w::T

    function MarkovState{T}(player_sequence::Vector{T}, H::Vector{T}, w::T) where T <: Integer
        new(player_sequence, H[player_sequence], w)
    end
end

MarkovState(player_sequence, H, w::T) where T <: Integer = MarkovState{T}(player_sequence, H, w)

# generates all possible markov states for a given n
function markov_states(n, H)
    # initial player sequence, e.g. abcd or 1234
    canonical_sequence = 1:n
    # all permutations of player sequences
    p_seqs = permutations(canonical_sequence)
    # amount of times the Mario Kart Master has won
    w_seq = 0:(n-2)

    vec([MarkovState(p_seq, H, w) for p_seq in p_seqs, w in w_seq])
end

markov_states (generic function with 1 method)

In [6]:
# abilities of the players(indexed in order)
H = [1, 1, 1, 1] # H is for 'habilidade'
n = length(H) # number of players
MS = markov_states(n, H)

# Transition probability rule
# Tells the probability of the state transition s(i) -> s(j)
function transition_prob(MS, i, j)
    if i == j
        0
    else
        # Get the actual states
        s1, s2 = MS[i], MS[j]

        p1 = s1.player_sequence
        p2 = s2.player_sequence
        
        master_s1 = p1[1] 
        master_s2 = p2[1] 

        challenger = p1[2]
        loser = last(p2)

        # Get the abilities of the players in each state
        x = s1.ability_sequence[1] # master's ability
        y = s1.ability_sequence[2] # challenger's ability

        # The internal ordering of the sequence must remain the same
        # otherwise, there will be no state transition
        if p1[3:end] != p2[2:end-1]
            0
        elseif master_s1 == master_s2 && challenger == loser && s2.w == (s1.w + 1)
            # Probability that the master will keep his title
            # Note: when this state transition occurs, the new state
            # has w += 1, i.e. the amount of times the master has won is increased by 1
            1 - prob_lose(x, y)^2
        elseif master_s2 == challenger && master_s1 == loser && s2.w == 0
            # Probability that the challenger will become the new master
            # Note: when this state transition occurs, the new state
            # has w = 0, i.e. the amount of times the master has won is reset to 0
            prob_lose(x, y)^2
        else
            0
        end
    end
end

transition_prob (generic function with 1 method)

In [7]:
# All indexes generated by the markov states
# So that we can create a |MS| × |MS| matrix of 
# state transition probabilities
ps = Array(1:length(MS))
q(a, b) = transition_prob(MS, a, b)
Q = q.(ps, ps')

72×72 Matrix{Real}:
  0     0     0     0     0     0    …  0  0  0  0  0  0  0  0  0  0  0  0
  0     0     0     0     0     0       0  0  0  0  0  0  0  0  0  0  0  0
  0     0     0     0     0     0       0  0  0  0  0  0  0  0  0  0  0  0
  0     0     0     0     0     0       0  0  0  0  0  0  0  0  0  0  0  0
  0     0     0     0     0     0       0  0  0  0  0  0  0  0  0  0  0  0
  0     0     0     0     0     0    …  0  0  0  0  0  0  0  0  0  0  0  0
  0     0     0    1//4   0     0       0  0  0  0  0  0  0  0  0  0  0  0
  0     0     0     0     0    1//4     0  0  0  0  0  0  0  0  0  0  0  0
  0     0     0     0     0     0       0  0  0  0  0  0  0  0  0  0  0  0
  0     0     0     0     0     0       0  0  0  0  0  0  0  0  0  0  0  0
  0     0     0     0     0     0    …  0  0  0  0  0  0  0  0  0  0  0  0
  0     0     0     0     0     0       0  0  0  0  0  0  0  0  0  0  0  0
  0    1//4   0     0     0     0       0  0  0  0  0  0  0  0  0  0  0  0
  ⋮  

In [8]:
function final_transition_prob(MS, n, i, j)
    # markov state(row)
    s = MS[i]
    master = s.player_sequence[1]
    # j indicates the final winner, which is the current master
    # (i.e. it is the absorption state in the markov chain)
    # n is the number of players
    if s.w == n - 2 && master == j
        # Get the abilities of the players in each state
        x = s.ability_sequence[1] # master's ability
        y = s.ability_sequence[2] # challenger's ability
        # Probability that the master will be the final winner
        1 - prob_lose(x, y)^2
    else
        0
    end
end

final_transition_prob (generic function with 1 method)

In [9]:
r(i, j) = final_transition_prob(MS, n, i, j)

# R: matrix which has the transition probabilities
# to the final states
R = r.(ps, (1:n)');

In [10]:
sum([Q R], dims=2)

72×1 Matrix{Real}:
 1//1
 1//1
 1//1
 1//1
 1//1
 1//1
 1//1
 1//1
 1//1
 1//1
 1//1
 1//1
 1//1
  ⋮
 1//1
 1//1
 1//1
 1//1
 1//1
 1//1
 1//1
 1//1
 1//1
 1//1
 1//1
 1//1

In [11]:
IQ = Float64.(I-Q)

# N: Fundamental matrix
N = inv(IQ);

In [12]:
# Absorption probabilities
NR = N*R;
NR

72×4 Matrix{Any}:
 0.506581   0.183106   0.163794   0.146519
 0.506581   0.183106   0.146519   0.163794
 0.506581   0.163794   0.183106   0.146519
 0.506581   0.146519   0.183106   0.163794
 0.506581   0.163794   0.146519   0.183106
 0.506581   0.146519   0.163794   0.183106
 0.183106   0.506581   0.163794   0.146519
 0.183106   0.506581   0.146519   0.163794
 0.163794   0.506581   0.183106   0.146519
 0.146519   0.506581   0.183106   0.163794
 0.163794   0.506581   0.146519   0.183106
 0.146519   0.506581   0.163794   0.183106
 0.183106   0.163794   0.506581   0.146519
 ⋮                                
 0.126645   0.0457765  0.78663    0.0409485
 0.126645   0.0409485  0.78663    0.0457765
 0.0457765  0.126645   0.78663    0.0409485
 0.0409485  0.126645   0.78663    0.0457765
 0.0457765  0.0409485  0.78663    0.126645
 0.0409485  0.0457765  0.78663    0.126645
 0.126645   0.0457765  0.0409485  0.78663
 0.126645   0.0409485  0.0457765  0.78663
 0.0457765  0.126645   0.0409485  0.78663
